## Dask with Slurm Cluster

In [ ]:
client.close()
cluster.close()

In [12]:
import dask, dask.distributed
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    # Dask worker size
    cores=8, memory='10GB',
    processes=1, # Dask workers per job
    walltime="00:10:00",
    #project="adaptive-cluster",
    queue="CPU",
    nanny=False, # No Nanny
    # Dask worker network and temporary storage
    interface='ib0', 
    local_directory='/tmp'
    )

client = dask.distributed.Client(cluster)

In [13]:
cluster

Dashboard: http://10.102.0.62:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.102.0.62:33845,Workers: 0
Dashboard: http://10.102.0.62:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [14]:
cluster.scale(6)

In [15]:
client

<Client: 'tcp://10.102.0.62:33845' processes=6 threads=48, memory=55.86 GiB>

In [16]:
print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p CPU
#SBATCH -n 1
#SBATCH --cpus-per-task=8
#SBATCH --mem=10G
#SBATCH -t 00:10:00

/gpfs/soma_fs/home/valerio/anaconda3/envs/neuron/bin/python -m distributed.cli.dask_worker tcp://10.102.0.62:33845 --nthreads 8 --memory-limit 9.31GiB --name dummy-name --no-nanny --death-timeout 60 --local-directory /tmp --interface ib0 --protocol tcp://



In [17]:
!squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
             96137       CPU dask-wor  valerio  R       0:26      1 somacpu005
             96138       CPU dask-wor  valerio  R       0:26      1 somacpu006
             96139       CPU dask-wor  valerio  R       0:26      1 somacpu007
             96140       CPU dask-wor  valerio  R       0:26      1 somacpu008
             96135       CPU dask-wor  valerio  R       0:27      1 somacpu010
             96136       CPU dask-wor  valerio  R       0:27      1 somacpu011


In [18]:
import numpy, dask.array

def calculate_pi(size_in_bytes, number_of_chunks):
    
    """Calculate pi using a Monte Carlo method."""
    
    array_shape = (int(size_in_bytes / 8 / 2), 2)
    chunk_size = (int(array_shape[0] / number_of_chunks), 2)
    
    # 2D random positions array using dask.array
    xy = dask.array.random.uniform(
        low=0.0, high=1.0, size=array_shape,
        # specify chunk size, i.e. task number
        chunks=chunk_size )
  
    xy_inside_circle = (xy ** 2).sum(axis=1) < 1 # boolean

    pi = 4 * xy_inside_circle.sum() / xy_inside_circle.size
    
    # start Dask calculation
    pi = pi.compute()

    print(f"\nfrom {xy.nbytes / 1e9} GB randomly chosen positions")
    print(f"   pi estimate: {pi}")
    print(f"   pi error: {abs(pi - numpy.pi)}\n")
    # display(xy)
    
    return pi

In [19]:
%time pi = calculate_pi(size_in_bytes=10_000_000_000, number_of_chunks=100) # 10 GB


from 10.0 GB randomly chosen positions
   pi estimate: 3.141541152
   pi error: 5.150158979327912e-05

CPU times: user 1.05 s, sys: 210 ms, total: 1.26 s
Wall time: 5.47 s


In [20]:
client.close()
cluster.close()